<a href="https://colab.research.google.com/github/stephenkati/Awesome-books/blob/master/Kiswahili-Mozilla-Common-Voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install huggingface_hub
!pip install librosa

In [ ]:
from datasets import load_dataset
train_data=load_dataset("mozilla-foundation/common_voice_11_0", "sw", split="train")
test_data=load_dataset("mozilla-foundation/common_voice_11_0", "sw", split="test")
validation_data=load_dataset("mozilla-foundation/common_voice_11_0", "sw", split="validation")

In [ ]:
train_data

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 26614
})

In [ ]:
test_data

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 10238
})

In [ ]:
validation_data

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 10233
})

In [ ]:
train_data=train_data.remove_columns(['client_id', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale','segment'])
test_data=test_data.remove_columns(['client_id', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale','segment'])
validation_data=validation_data.remove_columns(['client_id', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale','segment'])

In [ ]:
train_data

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 26614
})

In [ ]:
test_data

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 10238
})

In [ ]:
import re

# Modify the chars_to_remove_regex pattern to include the additional symbols
chars_to_remove_regex = r'[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\(\)\*\=\_`\[\]\/\*°ː’•…]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [ ]:
train_data= train_data.map(remove_special_characters)
test_data = test_data.map(remove_special_characters)
validation_data=validation_data.map(remove_special_characters)

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = train_data.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=train_data.column_names)
vocab_validate = validation_data.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=validation_data.column_names)
vocab_test = test_data.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=test_data.column_names)

In [ ]:
# Convert "vocab" column from each dataset to sets and union them
vocab_set_train = set(vocab_train["vocab"][0])
vocab_set_validate = set(vocab_validate["vocab"][0])
vocab_set_test = set(vocab_test["vocab"][0])

# Merge vocabularies
vocab_set = vocab_set_train | vocab_set_validate | vocab_set_test

# Convert the result back to a list
vocab_list = list(vocab_set)


vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict.pop('י', None)
vocab_dict["ụ"] = 45
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 'µ': 27,
 'á': 28,
 'â': 29,
 'ã': 30,
 'å': 31,
 'é': 32,
 'ë': 33,
 'í': 34,
 'ï': 35,
 'ñ': 36,
 'ó': 37,
 'ö': 38,
 'ø': 39,
 'ú': 40,
 'š': 41,
 'ū': 42,
 'ụ': 45}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

print(vocab_dict)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, 'µ': 27, 'á': 28, 'â': 29, 'ã': 30, 'å': 31, 'é': 32, 'ë': 33, 'í': 34, 'ï': 35, 'ñ': 36, 'ó': 37, 'ö': 38, 'ø': 39, 'ú': 40, 'š': 41, 'ū': 42, 'ụ': 45, '|': 0}


In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

46

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
!pip install accelerate==0.20.3

  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.23.0
    Uninstalling accelerate-0.23.0:
      Successfully uninstalled accelerate-0.23.0


In [ ]:
from datasets import load_metric, Audio
train_data = train_data.cast_column("audio", Audio(sampling_rate=16_000))
validation_data = validation_data.cast_column("audio", Audio(sampling_rate=16_000))
test_data = test_data.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
train_data[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/d0c515954317076cb4654c80caae844d8922490cb28ecb53f2df4b52ab7baa52/sw_train_0/common_voice_sw_28660554.mp3',
 'array': array([-8.52651283e-14, -1.35003120e-13, -9.94759830e-14, ...,
         4.05998435e-06, -4.27663326e-06, -2.08104029e-06]),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_data)-1)

print(train_data[rand_int]["sentence"])
ipd.Audio(data=train_data[rand_int]["audio"]["array"], autoplay=True, rate=16000)

wodini amekuta vitanda tupu na paka yuko juu


In [ ]:
rand_int = random.randint(0, len(train_data)-1)

print("Target text:", train_data[rand_int]["sentence"])
print("Input array shape:", train_data[rand_int]["audio"]["array"].shape)
print("Sampling rate:", train_data[rand_int]["audio"]["sampling_rate"])

Target text: inatakiwa kupanda mti sehemu iliyokatwa mti
Input array shape: (78336,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
train_data = train_data.map(prepare_dataset, remove_columns=train_data.column_names)
validation_data = validation_data.map(prepare_dataset, remove_columns=validation_data.column_names)
test_data = test_data.map(prepare_dataset, remove_columns=test_data.column_names)

Map:   0%|          | 0/26614 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10233 [00:00<?, ? examples/s]

Map:   0%|          | 0/10238 [00:00<?, ? examples/s]

In [ ]:
from datasets import concatenate_datasets

# Concatenate cv_swahili_train and cv_swahili_validate
combined_train_validate = concatenate_datasets([train_data, validation_data])

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
!pip install jiwer

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-48-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1925: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import create_repo
#create_repo("m-team_model")

In [ ]:
repo_name = "m-team_model"
#tokenizer.push_to_hub(repo_name)

In [ ]:
!pip install accelerate==0.20.1
!pip install transformers[torch]

  Using cached accelerate-0.20.1-py3-none-any.whl (227 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3


  Using cached accelerate-0.23.0-py3-none-any.whl (258 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.1
    Uninstalling accelerate-0.20.1:
      Successfully uninstalled accelerate-0.20.1


In [ ]:
from transformers import TrainingArguments

# Check if you are running on a CUDA-enabled device before enabling FP16
import torch
if torch.cuda.is_available():
    fp16_enabled = True
else:
    fp16_enabled = False
    print("CUDA device not available. Disabling FP16.")

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=fp16_enabled,  # Enable FP16 only if CUDA is available
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

CUDA device not available. Disabling FP16.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=combined_train_validate,
    eval_dataset=test_data,
    tokenizer=processor.feature_extractor,
)

In [1]:
trainer.train()

NameError: ignored